# Rating

In [43]:
import pandas as pd
import missingno as msno
import numpy as np

In [44]:
rating = pd.read_csv(r"../data/rating.csv")

In [45]:
rating.columns = rating.columns.str.lower() # en minuscular
rating.columns = rating.columns.str.strip() # quitar espacios en blanco
rating.head()

,userid,movieid,rating,timestamp
0,1,2,3.5,2005-04-02 23:53:47
1,1,29,3.5,2005-04-02 23:31:16
2,1,32,3.5,2005-04-02 23:33:39
3,1,47,3.5,2005-04-02 23:32:07
4,1,50,3.5,2005-04-02 23:29:40


In [46]:
# Total de registros
len(rating)

20000263

In [47]:
# Verificamos que no haya nulos
rating.isna().sum()

userid       0
movieid      0
rating       0
timestamp    0
dtype: int64

In [48]:
# Cambiar el tipo de datos `object` de timestamp a datetime
rating["timestamp"] = pd.to_datetime(rating["timestamp"])

In [49]:
rating.head()

,userid,movieid,rating,timestamp
0,1,2,3.5,2005-04-02 23:53:47
1,1,29,3.5,2005-04-02 23:31:16
2,1,32,3.5,2005-04-02 23:33:39
3,1,47,3.5,2005-04-02 23:32:07
4,1,50,3.5,2005-04-02 23:29:40


In [50]:
rating.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20000263 entries, 0 to 20000262
Data columns (total 4 columns):
 #   Column     Dtype         
---  ------     -----         
 0   userid     int64         
 1   movieid    int64         
 2   rating     float64       
 3   timestamp  datetime64[ns]
dtypes: datetime64[ns](1), float64(1), int64(2)
memory usage: 610.4 MB


In [51]:
# Adicionamos la columna `year` y llenar con el año del timestamp
rating["year"] =  rating['timestamp'].dt.year
rating["month"] =  rating['timestamp'].dt.month
rating.head()

,userid,movieid,rating,timestamp,year,month
0,1,2,3.5,2005-04-02 23:53:47,2005,4
1,1,29,3.5,2005-04-02 23:31:16,2005,4
2,1,32,3.5,2005-04-02 23:33:39,2005,4
3,1,47,3.5,2005-04-02 23:32:07,2005,4
4,1,50,3.5,2005-04-02 23:29:40,2005,4


In [52]:
# Obtenemos el rating promedio y el conteo general 
rating_movies_promedio = rating.groupby(["movieid"])["rating"].mean()
rating_movies_conteo = rating.groupby(["movieid"])["rating"].count()


In [53]:
# Promedio de rating por pelicula de forma global en todos los años
rating_movies_promedio.head()

movieid
1    3.921240
2    3.211977
3    3.151040
4    2.861393
5    3.064592
Name: rating, dtype: float64

In [54]:
# Conteo de rating por pelicula de forma global en todos los años
rating_movies_conteo.head()

movieid
1    49695
2    22243
3    12735
4     2756
5    12161
Name: rating, dtype: int64

In [55]:
# union de promeido y conteo
rating_promedio_conteo = pd.merge(rating_movies_promedio, rating_movies_conteo, on="movieid", how="left")

In [56]:
rating_promedio_conteo.head()

,rating_x,rating_y
movieid,,
1,3.921240,49695
2,3.211977,22243
3,3.151040,12735
4,2.861393,2756
5,3.064592,12161


In [57]:
# renombrar columnas
rating_promedio_conteo = rating_promedio_conteo.rename(
    columns={
        "rating_x": "rating_promedio",
        "rating_y": "rating_conteo"
    }
)

In [58]:
rating_promedio_conteo.head()

,rating_promedio,rating_conteo
movieid,,
1,3.921240,49695
2,3.211977,22243
3,3.151040,12735
4,2.861393,2756
5,3.064592,12161


In [59]:
# Es el total de registros
rating_promedio_conteo["rating_conteo"].sum()
# Esta OK

np.int64(20000263)

In [60]:
rating_promedio_conteo.head()

,rating_promedio,rating_conteo
movieid,,
1,3.921240,49695
2,3.211977,22243
3,3.151040,12735
4,2.861393,2756
5,3.064592,12161


In [61]:
# Guardado de tabla dimension usuarios temporal
rating_promedio_conteo.to_csv(r"../data/partial_rating.csv")

### Esto es para una tabla de dimensión `d_rating`
Contendra la pelicula agrupado por usuario, año y por mes

In [62]:
# Obtenemos el rating promedio por pelicula, año y mes
grupo = rating.groupby(["movieid","userid", "year","month"])["rating"]
rating_movies_year_month_promedio = grupo.mean()
rating_movies_year_month_conteo = grupo.count()

In [63]:
# Deberia ser una table dimension
rating_movies_year_month_promedio.head()

movieid  userid  year  month
1        3       1999  12       4.0
         6       1997  3        5.0
         8       1996  6        4.0
         10      1999  11       4.0
         11      2009  1        4.5
Name: rating, dtype: float64

In [64]:
rating_movies_year_month_conteo.head()

movieid  userid  year  month
1        3       1999  12       1
         6       1997  3        1
         8       1996  6        1
         10      1999  11       1
         11      2009  1        1
Name: rating, dtype: int64

In [65]:
# union de promeido y conteo
dim_rating = pd.merge(rating_movies_year_month_promedio, rating_movies_year_month_conteo, on=["movieid","userid", "year","month"], how="left")

In [66]:
dim_rating.head()

rating_x  rating_y
movieid userid year month                    
1       3      1999 12          4.0         1
        6      1997 3           5.0         1
        8      1996 6           4.0         1
        10     1999 11          4.0         1
        11     2009 1           4.5         1

In [67]:
# renombrar columnas
dim_rating = dim_rating.rename(
    columns={
        "rating_x": "promedio",
        "rating_y": "conteo"
    }
)

In [68]:
dim_rating.reset_index(inplace=True)

In [69]:
dim_rating

,movieid,userid,year,month,promedio,conteo
0,1,3,1999,12,4.0,1
1,1,6,1997,3,5.0,1
2,1,8,1996,6,4.0,1
3,1,10,1999,11,4.0,1
4,1,11,2009,1,4.5,1
...,...,...,...,...,...,...
20000258,131254,79570,2015,3,4.0,1
20000259,131256,79570,2015,3,4.0,1
20000260,131258,28906,2015,3,2.5,1
20000261,131260,65409,2015,3,3.0,1


In [70]:
dim_rating.head()

,movieid,userid,year,month,promedio,conteo
0,1,3,1999,12,4.0,1
1,1,6,1997,3,5.0,1
2,1,8,1996,6,4.0,1
3,1,10,1999,11,4.0,1
4,1,11,2009,1,4.5,1


In [71]:
dim_rating["ratingid"] = range(1, len(dim_rating) + 1)

In [72]:
dim_rating.set_index("ratingid", inplace=True)

In [73]:
dim_rating.head()

,movieid,userid,year,month,promedio,conteo
ratingid,,,,,,
1,1,3,1999,12,4.0,1
2,1,6,1997,3,5.0,1
3,1,8,1996,6,4.0,1
4,1,10,1999,11,4.0,1
5,1,11,2009,1,4.5,1


In [74]:
# Guardado de tabla de dimension
dim_rating.to_csv(r"../data/d_rating.csv")